In [1]:
import torch
from recbole.utils.case_study import full_sort_topk, full_sort_scores
from recbole.quick_start import load_data_and_model

In [2]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file="saved/[256, 128, 64, 32]_0001.pth",
)

02 Jan 14:13    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/epm
checkpoint_dir = saved/
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 600
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field

## Just for checking userID/ShiptoID

In [3]:
import pandas as pd

df = pd.read_parquet("raw_data/RatingMatrix.parquet")
df.head()

,ship_to_id,item_code,final_rating
0,EPM_34952,TCTLB,2.9
1,EPM_35444,TCTLB,2.7
2,EPM_3238950,TCTLB,1.9
3,EPM_136080,TCTLB,4.4
4,EPM_4393798,TCTLB,2.7


In [10]:
len(df)

69087

## Loop Here for each user

In [7]:
# uid_series = np.array([1, 2])  # internal user id series
# or you can use dataset.token2id to transfer external user token to internal user id
uid_series = dataset.token2id(dataset.uid_field, ["EPM_4393798"])
uid_series

array([5])

In [8]:
topk_score, topk_iid_list = full_sort_topk(
    uid_series, model, test_data, k=10, device=config["device"]
)

predict var: tensor([0.4907, 0.8607, 0.9237,  ..., 0.1594, 0.4886, 0.0747], device='cuda:0')


predict labels: tensor([0., 1., 1.,  ..., 0., 0., 0.], device='cuda:0')


In [9]:
print(topk_score)  # scores of top 10 items
# print(topk_iid_list)  # internal id of top 10 items
external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
print(external_item_list)  # external tokens of top 10 items
print()


# score = full_sort_scores(uid_series, model, test_data, device=config["device"])
# print(score)  # score of all items
# print(
#     score[0, dataset.token2id(dataset.iid_field, ["10137"])]
# )  # score of item ['242', '302'] for user '196'.


tensor([[0.9621, 0.9557, 0.9453, 0.9417, 0.9374, 0.9349, 0.9324, 0.9237, 0.9226,
         0.9193]], device='cuda:0')
[['TMECB' 'TNTPK' 'LKXLA' 'LKXOB' 'LTPRD' 'TSVNE' 'CTD15' 'TALNF' 'IM058'
  'LDPDA']]

